In [ ]:
from sklearn.metrics import f1_score
import numpy as np
import random
import os
import cv2 as ocv
import numpy as np
from sklearn.metrics import confusion_matrix
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split

random.seed(123)
np.random.seed(123)

C_global = [x for x in range(1, 10, 1)]
degree_global = [x for x in range(2, 10, 1)]
gamma_global = [0.0001, 0.001, 0.01]
kernel_global = ['linear', 'rbf', 'poly', 'sigmoid']

"""
DEFINE THE INITIAL POPULATION WITH THE HYPERPARAMETERS NEEDED TO OPTIMIZE
"""
def initialize_Population(numParents):
  C = np.empty([numParents, 1], dtype=np.uint8)
  degree = np.empty([numParents, 1], dtype=np.uint8)
  gamma = np.empty([numParents, 1], dtype=np.float64)
  kernel = np.empty([numParents, 1], dtype=np.uint8)
  for i in range(numParents):
    C[i] = random.randint(1, 10)
    degree[i] = random.randint(2, 10)
    gamma[i] = gamma_global[random.randint(0, 2)]
    kernel[i] = random.randint(0, 3)

  population = np.concatenate((C, degree, gamma, kernel), axis=1)
  return population

"""
DEFINE THE FITNESS FUNCTION AS THE F1_SCORE
"""
def f1_Fitness_Score(y_true, y_pred):
  fitness = round((f1_score(y_true, y_pred, average='weighted')), 4)
  return fitness

"""
DEFINE THE TRAINING ALGORITHM WITH THE DIFFERENT HYPERPARAMETERS GENERATED
BY THE GENETIC ALGORITHM
"""
def train_Hyperparameter(population, model, xTrain, yTrain, xTest, yTest):
  fScore = []
  for i in range(population.shape[0]):
    param = {
        'C': population[i][0],
        'degree': population[i][1],
        'gamma': population[i][2],
        'kernel': kernel_global[int(population[i][3])]
    }
    model.fit(xTrain, yTrain)
    preds = model.predict(xTest)
    fScore.append(f1_Fitness_Score(yTest, preds))
  return fScore

"""
DEFINE THE PARENT SELECTION FUNCTION FOR MATING THE BEST PARENTS TO
PRODUCE THE BEST POSSIBLE OFFSPRING CHROMOSOMES
"""
def new_parent_selection(population, fitness, numParents):
  selectedParents = np.empty((numParents, population.shape[1]))
  for parentId in range(numParents):
        bestFitnessId = np.where(fitness == np.max(fitness))
        bestFitnessId  = bestFitnessId[0][0]
        selectedParents[parentId, :] = population[bestFitnessId, :]
        fitness[bestFitnessId] = -1 
  return selectedParents


"""
DEFINE THE CROSSOVER OPERATOR.
FOR THIS, WE WILL USE THE UNIFORM CROSSOVER
"""
def uniform_crossover(parents, childrenSize):
    crossoverPointIndex = np.arange(0, np.uint8(childrenSize[1]), 1, dtype= np.uint8) 
    crossoverPointIndex1 = np.random.randint(0, np.uint8(childrenSize[1]), np.uint8(childrenSize[1]/2)) 
    crossoverPointIndex2 = np.array(list(set(crossoverPointIndex) - set(crossoverPointIndex1))) 
    
    children = np.empty(childrenSize)

    for i in range(childrenSize[0]):
        parent1_index = i%parents.shape[0]

        parent2_index = (i+1)%parents.shape[0]

        children[i, crossoverPointIndex1] = parents[parent1_index, crossoverPointIndex1]

        children[i, crossoverPointIndex2] = parents[parent2_index, crossoverPointIndex2]
    return children

"""
DEFINE THE MUTATION OPERATOR.
FOR HYPERPARAMETER SELECTION, WE USE A MODIFIED 'BIT FLIP' OPERATOR
INSTEAD OF FLIPPING BITS, WE SELECT A NEW VALUE WITHIN THE ACCEPTABLE RANGE
"""
def mutation(crossover, numParameters):
  minmaxValue = np.zeros((numParameters, 2))

  minmaxValue[0:] = [1, 10]
  minmaxValue[1:] = [2, 10]
  minmaxValue[2:] = [0.0001, 0.01]
  minmaxValue[3:] = [0,3]

  mutationVal = 0
  parameterSelect = np.random.randint(0, 3, 1)

  if parameterSelect == 0:
    mutationVal = np.random.randint(1, 10)
  if parameterSelect == 1:
    mutationVal = np.random.randint(2, 10)
  if parameterSelect == 2:
    mutationVal = gamma_global[random.randint(0,2)]
  if parameterSelect == 3:
    mutationVal = kernel_global[random.randint(0,3)]
  
  for idx in range(crossover.shape[0]):
        crossover[idx, parameterSelect] = crossover[idx, parameterSelect] + mutationVal
        if(crossover[idx, parameterSelect] > minmaxValue[parameterSelect, 1]):
            crossover[idx, parameterSelect] = minmaxValue[parameterSelect, 1]
        if(crossover[idx, parameterSelect] < minmaxValue[parameterSelect, 0]):
            crossover[idx, parameterSelect] = minmaxValue[parameterSelect, 0]    
  return crossover

In [ ]:
#FIRST MOUNT DRIVE TO THE COLAB NOTEBOOK
!pip install unrar
!unrar x drive/MyDrive/xray_dataset_covid19.rar

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/

UNRAR 5.50 freeware      Copyright (c) 1993-2017 Alexander Roshal

Cannot open drive/MyDrive/xray_dataset_covid19.rar
No such file or directory
No files to extract


In [ ]:
np.random.seed(123)
from google.colab import drive
drive.mount('/content/drive')
!unrar x drive/MyDrive/xray_dataset_covid19.rar
dir = 'xray_dataset_covid19/train'

classes = ['NORMAL','PNEUMONIA']

Data = []
Lables = []
for category in os.listdir(dir):
    newPath = os.path.join(dir,category)
    for img in os.listdir(newPath):
        img_path = os.path.join(newPath,img)
        if 'Thumbs.db' not in img_path:
            Data.append((np.array(ocv.resize(ocv.imread(img_path,0),(100,100))).flatten())/255)
            Lables.append(classes.index(category))

print(np.shape(Data))
X_train, X_test,Y_train, Y_test = train_test_split(Data,Lables,test_size=0.33)
model = SVC()

numParents = 25
numParentsMating = 24
numParameters = 4
numGenerations = 20

popSize = (numParents, numParameters)

population = initialize_Population(numParents)

fitnessHistory = np.empty([numGenerations+1, numParents])

populationHistory = np.empty([(numGenerations+1)*numParents, numParameters])

populationHistory[0:numParents, :] = population

for gen in range(numGenerations):
  print("This is number %s generation" % (gen))

  fitnessValue = train_Hyperparameter(population, model, X_train, Y_train, X_test, Y_test)
  fitnessHistory[gen, :] = fitnessValue

  print('Best F1 score in the this iteration = {}'.format(np.max(fitnessHistory[gen, :])))

  parents = new_parent_selection(population, fitnessValue, numParentsMating)

  children = uniform_crossover(parents, (popSize[0] - parents.shape[0], numParameters))

  children_mutated = mutation(children, numParameters)

  population[0:parents.shape[0], :] = parents
  population[parents.shape[0]:, :] = children_mutated 
  
  populationHistory[(gen+1)*numParents : (gen+1)*numParents+ numParents , :] = population 


fitness = train_Hyperparameter(population, model, X_train, Y_train, X_test, Y_test)
fitnessHistory[gen+1, :] = fitness

bestFitnessIndex = np.where(fitness == np.max(fitness))[0][0]

print("Best fitness is =", fitness[bestFitnessIndex])

print("Best parameters are:")
print("C = ", population[bestFitnessIndex][0])
print("degree = ", population[bestFitnessIndex][1])
print("gamma = ", population[bestFitnessIndex][2])
print("kernel = ", kernel_global[int(population[bestFitnessIndex][3])])



Mounted at /content/drive

UNRAR 5.50 freeware      Copyright (c) 1993-2017 Alexander Roshal


Extracting from drive/MyDrive/xray_dataset_covid19.rar

Creating    xray_dataset_covid19                                      OK
Creating    xray_dataset_covid19/test                                 OK
Creating    xray_dataset_covid19/test/NORMAL                          OK
Extracting  xray_dataset_covid19/test/NORMAL/nor.jpeg                      0%  OK 
Extracting  xray_dataset_covid19/test/NORMAL/NORMAL2-IM-0035-0001.jpeg       0%  OK 
Extracting  xray_dataset_covid19/test/NORMAL/NORMAL2-IM-0052-0001.jpeg       0%  OK 
Extracting  xray_dataset_covid19/test/NORMAL/NORMAL2-IM-0058-0001.jpeg       0%  OK 
Extracting  xray_dataset_covid19/test/NORMAL/NORMAL2-IM-0059-0001.jpeg       0%  1%  OK 
Extracting  xray_dataset_covid19/test/NORMAL/NORMAL2-IM-0072-0001.jpeg       1%  OK 
Extracting  xray_dataset_covid19/test/NORMAL/NORMAL2-IM-0073